<a href="https://colab.research.google.com/github/ASJIDALI/Projects/blob/main/Product_Sales_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Product Sales Analysis**


> Monthly Profit Tracking: Calculates and logs each product's profit from the start of the month to the current day, enabling continuous performance monitoring.
> Total Profit: Aggregates total profit for each product, offering a long-term view of profitability.
> Current Stock Analysis: Displays up-to-date stock levels, helping to identify products with potential stock shortages or surpluses.
> Pricing Insights: Tracks the highest and lowest prices offered for each product within the selected interval, assisting in pricing strategy refinement.
> Procurement Actions: Identifies recommended procurement actions based on stock levels and profitability.
> Sales Verification Status: Verifies sales records for accuracy and integrity, flagging items for review by the sales team if discrepancies are detected.

> **Usage:**
> This analysis can be implemented as part of an e-commerce sales dashboard, providing business teams with critical insights on product performance and inventory needs. Users can apply this project to any sales dataset with similar fields to gain a clearer understanding of daily product trends and long-term profitability.
In sales data , you can see the columns
 ItemCode : unique code for the specific item

*   ItemCode : unique code for the specific item
*   ItemNAme : Name of the product
*   Brand : Brand of the product
*   Category ,Sub Category1, Sub Category2, Sub Categoory3 : This as category and the sub categories of the products , here we are using sub category 3 for the analysis.
*   Date : Date of the transaction happened
*   Sales Value: it is the regualr price of the product
*   Profit: Profit from the product from that transaction
*   Quantity : Quantity sold in that transaction for that specific product.
*   Cost Value: Cost of the product
*   ItemGroup: used to identify in which category product belong to , such as device , accessories, health ,etc
*   Type:POS sale means its a retail sale and Corp Sale means its a wholesale sale , serivce means transaction for a particular service for any specific product.
*   'Invoice Number', 'Customer', 'Sales Employee': These are not mainly used in this case of analysis , but we can used it to find the loyal customers and best perforemed staff , also useful to identify the recommendation a specific product that can be useful in a ecommerce site


# Data Loading ,Engineering,Cleaning


In [1]:
# from date and to date for make a intreval to know how much sales happened in each month in that interval
from datetime import date, timedelta
fromd=1
tod=(date.today() - timedelta(days=1)).day

In [2]:
# importing required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', '{:.2f}'.format)
import warnings
warnings.filterwarnings('ignore')

In [3]:
# to mail to the representives , importing libraries for mail
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders
import os

In [4]:
# df1 is sales from january to previous month and curr is the current month sales
df1=pd.read_csv('https://docs.google.com/spreadsheets/d/1W-KrmwpF9gaLeMq9y15JxxQlnTyy-kYaQEr4-y8Bgc4/pub?gid=1294612192&single=true&output=csv', low_memory=False)
cur=pd.read_csv('https://docs.google.com/spreadsheets/d/1q4cXYcpXsdJkR3DHOXBjPkFewmdQb8w_sR4toEIwC5w/pub?gid=0&single=true&output=csv', low_memory=False)

In [5]:
# cleaning and give appropriate data type and concate previous and current sales report
cur['Date'] = pd.to_datetime(cur['Date'], format='%d/%m/%Y')
df=pd.concat([df1,cur],ignore_index=True)
df['Date'] = pd.to_datetime(df['Date'])
df.Profit=df.Profit.astype(float)

In [7]:
df.head()

,Type,Invoice Number,Customer,Sales Employee,Date,mont,Category,Sub Category1,Sub Category2,Sub Category3,Sub Category4,ItemCode,ItemName,ItemGroup,Brand,Quantity,Warehouse,Cost Value,Sales Value,Profit,Margin %
0,Service,24000001,DENNIS,-No Sales Employee-,2024-01-01,January,SERVICE,HARDWARE,NaN,NaN,NaN,DS,DISPLAY SERVICE,CLICK N FIX,NaN,1,EZDAN WAKRAH-AL ANEES ELECTRONICS,28.00,140.00,112.00,80.00
1,Service,24000002,WAHEED,-No Sales Employee-,2024-01-01,January,SERVICE,HARDWARE,NaN,NaN,NaN,HS,HARDWARE SERVICE,CLICK N FIX,NaN,1,EZDAN WAKRAH-AL ANEES ELECTRONICS,156.09,350.00,193.91,55.40
2,Service,24000003,abrar ali,-No Sales Employee-,2024-01-01,January,SERVICE,HARDWARE,NaN,NaN,NaN,OEMD,OEM Display Replacement,CLICK N FIX,NaN,1,AL WATAN-AL ANEES TECHNOLOGY,558.00,660.00,102.00,15.45
3,Service,24000004,abid,-No Sales Employee-,2024-01-01,January,SERVICE,HARDWARE,NaN,NaN,NaN,HS,HARDWARE SERVICE,CLICK N FIX,NaN,1,AL WATAN-AL ANEES TECHNOLOGY,110.00,490.00,380.00,77.55
4,Service,24000005,joenard,-No Sales Employee-,2024-01-01,January,SERVICE,HARDWARE,NaN,NaN,NaN,SGDR,S+ Grade Display Replacement,CLICK N FIX,NaN,1,AL WATAN-AL ANEES TECHNOLOGY,130.00,270.00,140.00,51.85


In [ ]:
# loading stock report and make requied changes
stock=pd.read_csv("https://docs.google.com/spreadsheets/d/1q4cXYcpXsdJkR3DHOXBjPkFewmdQb8w_sR4toEIwC5w/pub?gid=168637289&single=true&output=csv",low_memory=False)
stock.rename(columns={'Item No.':'ItemCode'},inplace=True)
stock['ItemCode']=stock['ItemCode'].astype(str)
stock['In Stock']=stock['In Stock'].astype(int)

In [ ]:
# a display sheet name file include only  current trend items, used to give attractive offers
disply=pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vQJLZvqDq7RSNJfNGzlNAl0mBYXOZE4umutSeuqm_7ekWNvOOdZu-3BQrF-PLE2dgg8oNKdIL43TP3C/pub?gid=1195549924&single=true&output=csv').iloc[:,[1,0]]
disply.rename(columns={'Item No.':'ItemCode'},inplace=True)
disply['ItemCode']=disply['ItemCode'].astype(str)
disply.dropna(inplace=True)

In [ ]:
# product list published in ecommerce site
Ecom=pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vQJLZvqDq7RSNJfNGzlNAl0mBYXOZE4umutSeuqm_7ekWNvOOdZu-3BQrF-PLE2dgg8oNKdIL43TP3C/pub?gid=475798898&single=true&output=csv',low_memory=False)
Ecom.dropna(inplace=True)
Ecom.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10658 entries, 0 to 10659
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   ItemCode  10658 non-null  object
 1   Ecom      10658 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 249.8+ KB


In [ ]:
# loading data of procurment
prcmnt=pd.read_csv('https://docs.google.com/spreadsheets/d/1q4cXYcpXsdJkR3DHOXBjPkFewmdQb8w_sR4toEIwC5w/pub?gid=1919807903&single=true&output=csv',low_memory=False)
prcmnt.rename(columns={'ITEM CODE':'ItemCode'},inplace=True)
prcmnt['ItemCode']=prcmnt['ItemCode'].astype(str)
prcmnt.dropna(subset=['ItemCode','Date'],inplace=True)
prcmnt['Date'].replace({'8/7/2024':'07-08-2024','8/15/2024':'15-08-2024','8/20/2024':'20-08-2024'},inplace=True)
prcmnt['Date']=pd.to_datetime(prcmnt['Date'],format='%d-%m-%Y')
prcmnt.rename(columns={'Date':'P.Date'},inplace=True)
prcmnt.rename(columns={'Sales Team Confirmation':'Sales'},inplace=True)
# prcmnt['Sales'].replace({'True':'Yes','False':'No'},inplace=True)
prcmnt['ACTION'].fillna('_',inplace=True)
prcmnt['REMARK'].fillna('_',inplace=True)
prcmnt['Sales'].fillna('_',inplace=True)
prcmnt.Sales.value_counts()

,count
Sales,
True,8781
False,1923
_,6


In [ ]:
prcmnt1=prcmnt.groupby('ItemCode').last().reset_index().copy()

In [ ]:
# extract latest name , subcategory, brand name used for the specific product
latest_name=df.groupby('ItemCode').last().reset_index()[['ItemCode','ItemName']]
latest_cat=df.groupby('ItemCode').last().reset_index()[['ItemCode','Sub Category3']]
latest_brand=df.groupby('ItemCode').last().reset_index()[['ItemCode','Brand']]

In [ ]:
# desfine function to clean and filter stock report

def stock1(stock):
  warehouses = ["MANSOURA-AL ANEES ELECTRONICS",
      "AL WATAN-AL ANEES TECHNOLOGY",
      "BARWA LULU-AL ANEES ELECTRONICS",
      "EZDAN WAKRAH-AL ANEES ELECTRONICS",
      "GRANDMALL-AL ANEES ELECTRONICS",
      "GULF MALL AL ANEES ELECTRONICS CO WLL",
      "PLAZA MALL 220-AL ANEES ELECTRONICS",
      "RAYYAN-AL ANEES COMMUNICATION",
      "SHAHANIYA-AL ANEES ELECTRONICS",
      "CITY CENTER AL ANEES ELECTRONICS",
      "E-COMMERCE",
      "METRO ALSADD AL ANEES",
      "METRO MANSOURA AL-ANEES ELECTRONICS CO WLL",
      "ABU SIDRA MALL AL ANEES",
      "PHONE BOOTH DOHA FESTIVAL CITY",
      "PHONE BOOTH MOQ SHOP1",
      "PHONE BOOTH MSHEIREB DOWNTOWN",
      "PHONE BOOTH PLACE VENDOME MALL",
      "CENTRAL WAREHOUSE-AL ANEES ELECTRONICS",
      "AL ANEES CENTRAL WAREHOUSE BIRKAT AL AWAMER"]
  fstock = stock[(stock['WhsName'].isin(warehouses))&(stock['ItmsGrpNam'].isin(["ACCESSORIES", "CUSTOMIZED PRODUCTS", "DEVICES", "HEALTH & BEAUTY", "HOME & APPLIANCES", "LIFESTYLES"]))]
  pivot_table = fstock.pivot_table(index=['ItemCode','Item Description','Brand'],values='In Stock',aggfunc='sum',fill_value=0)
  pivot_table
  return pivot_table.reset_index()

In [ ]:
# calling the defined stock function
stock12=stock1(stock).rename(columns={'Item Description':'ItemName'})[['ItemCode','In Stock']]
# to know how much columns in cleaned stock report
stock12.shape

(7119, 2)

In [ ]:
# defining function for extraction required data from sales report , we need item code , item name , product category, product brand, profit in each month,
# in a interval from beggining of the month to the current day to analyse the profit status of each month in the current situation
def sales_summary_by_day_range(df, from_date, to_date):
    df['Date'] = pd.to_datetime(df['Date'])
    warehouses = ["MANSOURA-AL ANEES ELECTRONICS",
    "AL WATAN-AL ANEES TECHNOLOGY",
    "BARWA LULU-AL ANEES ELECTRONICS",
    "EZDAN WAKRAH-AL ANEES ELECTRONICS",
    "GRANDMALL-AL ANEES ELECTRONICS",
    "GULF MALL AL ANEES ELECTRONICS CO WLL",
    "PLAZA MALL 220-AL ANEES ELECTRONICS",
    "RAYYAN-AL ANEES COMMUNICATION",
    "SHAHANIYA-AL ANEES ELECTRONICS",
    "CITY CENTER AL ANEES ELECTRONICS",
    "E-COMMERCE",
    "METRO ALSADD AL ANEES",
    "METRO MANSOURA AL-ANEES ELECTRONICS CO WLL",
    "ABU SIDRA MALL AL ANEES",
    "PHONE BOOTH OLD DOHA PORT"
    "PHONE BOOTH DOHA FESTIVAL CITY",
    "PHONE BOOTH MOQ SHOP1",
    "PHONE BOOTH MSHEIREB DOWNTOWN",
    "PHONE BOOTH PLACE VENDOME MALL",
    "METRO LUSAIL AL ANEES"]
    filtered_df = df[(df['Date'].dt.day >= from_date) & (df['Date'].dt.day <= to_date)&(df['Type'].isin(['POS Sale', 'POS Return'])) &
          (df['Warehouse'].isin(warehouses))].copy()
    filtered_df['Month'] = filtered_df['Date'].dt.strftime('%B')
    pivot_table = filtered_df.pivot_table(
        index=['ItemCode'],
        columns='Month',
        values='Profit',
        aggfunc='sum',
        fill_value=0
    )
    pivot_table = pivot_table.reindex(columns=['January', 'February', 'March', 'April', 'May', 'June', 'July','August','September','October'], fill_value=0)
    pivot_table['Total'] = pivot_table.sum(axis=1)
    result = pivot_table.reset_index().sort_values(by='October', ascending=False)
    result.rename(columns={'January': 'Jan', 'February': 'Feb', 'March': 'Mar', 'April': 'Apr', 'May': 'May', 'June': 'Jun', 'July': 'Jul', 'August': 'Aug','September':'Sept',
                           'October':'Oct'}, inplace=True)
    return result

# defining fuction to highlight lowest and highest profit from the months , lowest will be coloured as red and highest will be colloured as green

def highlight_min_max(row):
    styles = []
    numeric_cols = row.index[4:-10]  # Select columns from 'January' to 'August'
    min_col = row.loc[numeric_cols].idxmin()
    max_col = row.loc[numeric_cols].idxmax()
    for col in row.index:
        if col in ['ItemCode', 'ItemName', 'Total']:
            styles.append('')
        elif col == min_col:
            styles.append('color: red; font-weight: bold;')  # Highlight minimum
        elif col == max_col:
            styles.append('color: green; font-weight: bold;') # Highlight maximum
        else:
            styles.append('')
    return styles

In [ ]:
# defin fuction to find maximum and minimum sale price offered for a product in the particular time period
warehouses = ["MANSOURA-AL ANEES ELECTRONICS",
    "AL WATAN-AL ANEES TECHNOLOGY",
    "BARWA LULU-AL ANEES ELECTRONICS",
    "EZDAN WAKRAH-AL ANEES ELECTRONICS",
    "GRANDMALL-AL ANEES ELECTRONICS",
    "GULF MALL AL ANEES ELECTRONICS CO WLL",
    "PLAZA MALL 220-AL ANEES ELECTRONICS",
    "RAYYAN-AL ANEES COMMUNICATION",
    "SHAHANIYA-AL ANEES ELECTRONICS",
    "CITY CENTER AL ANEES ELECTRONICS",
    "E-COMMERCE",
    "METRO ALSADD AL ANEES",
    "METRO MANSOURA AL-ANEES ELECTRONICS CO WLL",
    "ABU SIDRA MALL AL ANEES",
    "PHONE BOOTH OLD DOHA PORT"
    "PHONE BOOTH DOHA FESTIVAL CITY",
    "PHONE BOOTH MOQ SHOP1",
    "PHONE BOOTH MSHEIREB DOWNTOWN",
    "PHONE BOOTH PLACE VENDOME MALL",
    "METRO LUSAIL AL ANEES"]
df2=df[(df['Quantity']==1)&(df['Type']=='POS Sale')&(df['Sales Value']!=0)]
max_s=df2.groupby(['ItemCode'])['Sales Value'].max()
min_s=df2.groupby(['ItemCode'])['Sales Value'].min()

**Sales Latest Updated Date**:



In [ ]:
# to know last updated date of sales data
print('Latest Update')
df['Date'].tail(1).iloc[0].strftime('%Y-%m-%d')

Latest Update


'2024-10-30'

### Here merging and data engineering will be done


In [ ]:
summary_df = sales_summary_by_day_range(df, fromd, tod)
summary_df=summary_df.merge(latest_brand,on='ItemCode',how='left')
summary_df=summary_df.merge(latest_name,on='ItemCode',how='left')
summary_df=summary_df.merge(stock12,on=('ItemCode'),how='outer')
summary_df=summary_df.merge(latest_cat,on='ItemCode',how='left')
summary_df=summary_df.merge(disply,on='ItemCode',how='left')
summary_df.rename(columns={'Shelf Code':'DS'},inplace=True)
summary_df['DS']=summary_df['DS'].fillna('Not mentioned')
summary_df=summary_df.merge(Ecom,on='ItemCode',how='left')
summary_df['Ecom']=summary_df['Ecom'].fillna('Not listed')
summary_df['Ecom'].replace({1.0:'Published'},inplace=True)
summary_df=summary_df.merge(min_s,on='ItemCode',how='left')
summary_df=summary_df.merge(max_s,on='ItemCode',how='left')
summary_df.rename(columns={'Sales Value_x':'min_s','Sales Value_y':'max_s'},inplace=True)
summary_df['Sub Category3']=summary_df['Sub Category3'].fillna('No Category')
summary_df=summary_df.iloc[:,[0,13,-5,12,1,2,3,4,5,6,7,8,9,10,11,-6,-4,-3,-2,-1]]
summary_df=summary_df.merge(prcmnt1,on='ItemCode',how='left')

In [ ]:
# to know the columns in the summary_df data
summary_df.columns

Index(['ItemCode', 'ItemName', 'Sub Category3', 'Brand', 'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
       'Jul', 'Aug', 'Sept', 'Oct', 'Total', 'In Stock', 'DS', 'Ecom', 'min_s', 'max_s', 'P.Date',
       'Sales', 'ACTION', 'REMARK'],
      dtype='object')

In [ ]:
# filling the data , for the item name not in sales report , but only in stock report , only not saled yet data will be replaced like this
for index, row in summary_df.iterrows():
    if pd.isna(row['ItemName']):
        item_code = row['ItemCode']
        # Look up item name from stock report
        item_name = stock[stock['ItemCode'] == item_code]['Item Description'].values[0] if item_code in stock['ItemCode'].values else 'Not Found'
        summary_df.at[index, 'ItemName'] = item_name
        # Look up brand from stock report
        brand = stock[stock['ItemCode'] == item_code]['Brand'].values[0] if item_code in stock['ItemCode'].values else 'Not Found'
        summary_df.at[index, 'Brand'] = brand

In [ ]:
summary_df[['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun','Jul', 'Aug','Sept','Oct','Total','In Stock','min_s','max_s']]=summary_df[['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun','Jul', 'Aug','Sept','Oct','Total', 'In Stock','min_s','max_s']].fillna(0)
summary_df['In Stock']=summary_df['In Stock'].astype(int)
summary_df['Brand']=summary_df['Brand'].fillna('Not found')
summary_df.rename(columns={'ItemName':f'ItemName ({fromd}-{tod})'},inplace=True)

In [ ]:
summary_df=summary_df[(~summary_df[f'ItemName ({fromd}-{tod})'].str.contains('GIFT'))&(~summary_df[f'ItemName ({fromd}-{tod})'].str.contains('LDU'))&(~summary_df[f'ItemName ({fromd}-{tod})'].str.contains('AS IS'))&(~summary_df[f'ItemName ({fromd}-{tod})'].str.contains('USED'))]

In [ ]:
summary_df.sort_values(by='Oct',ascending=False,inplace=True)

In [ ]:
summary_df.head(5)

,ItemCode,ItemName (1-30),Sub Category3,Brand,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sept,Oct,Total,In Stock,DS,Ecom,min_s,max_s,P.Date,Sales,ACTION,REMARK
1826,104102299,APPLE IPHONE 16 PRO MAX 256GB - DESERT TITANIUM,IOS PHONES,APPLE,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,85255.28,85255.28,0,ONGOING,Published,5399.00,6069.00,NaT,NaN,NaN,NaN
133,104100155,APPLE IPHONE 15 PRO MAX 256GB - NATURAL TITANIUM(MU793),IOS PHONES,APPLE,16669.42,40681.33,38319.40,24780.36,2475.09,11732.59,25565.58,18623.60,2361.00,25804.26,207012.63,55,ONGOING,Published,4039.00,4999.00,2024-09-24,True,_,_
695,104100910,SAMSUNG GALAXY S24 ULTRA 5G 12/256GB - TITANIUM GRAY,ANDROID PHONES,SAMSUNG,4112.00,10018.19,8993.60,6412.00,2410.33,10004.22,12769.15,15900.15,20062.00,22109.00,112790.64,46,ONGOING,Published,3260.00,5149.00,2024-09-05,True,Available Now,_
692,104100907,SAMSUNG GALAXY S24 ULTRA 5G 12/256GB - TITANIUM BLACK,ANDROID PHONES,SAMSUNG,3595.00,15473.24,8907.98,6820.18,2366.13,8973.35,12320.05,16057.00,16418.00,19079.00,110009.93,52,ONGOING,Published,3279.00,5149.00,2024-08-19,True,_,_
3687,105101967,SAMSUNG USB-C 25W POWER ADAPTER (EP-T2510) - WHITE,USB-C,SAMSUNG,0.00,0.00,0.00,0.00,3432.00,14097.00,14004.00,16775.50,16580.00,15594.50,80483.00,453,ONGOING,Published,49.00,59.00,2024-09-10,False,_,_


In [ ]:
# apply the defined formatting function to identify maximum and minimum value easly
numeric_columns=['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sept','Oct','Total','min_s','max_s']
styled_df = summary_df.style.apply(highlight_min_max, axis=1).format("{:.2f}", subset=numeric_columns).format("{:.0f}", subset='In Stock')

In [ ]:
# this will dislay in the colour formated version
display(styled_df)

# Which are lowest Saled product in August with respect to remaining month in 2024

In [ ]:
# summary_df1=summary_df.copy()
# summary_df1['Min_Value'] = summary_df1.loc[:, 'Jan':'Jul'].min(axis=1)
# summary_df1['Min_Month'] = summary_df1.loc[:, 'Jan':'Jul'].idxmin(axis=1)
# min_august_products = summary_df1[summary_df1['Min_Value'] > summary_df1['Aug']]
# result_df = min_august_products[['ItemCode',f'ItemName ({fromd}-{tod})', 'Min_Month', 'Min_Value', 'Aug']]


**To Download extraxted  insight**

In [ ]:
styled_df.to_excel(f'Product Analysis {fromd}-{tod}.xlsx', index=False)

# defining function to mail the data

In [ ]:
# @title
def send_email(sender_email, receiver_email, cc_email, subject, body, attachment_path):
  """Sends an email with an optional attachment."""
  msg = MIMEMultipart()
  msg['From'] = sender_email
  msg['To'] = ', '.join(receiver_email)
  msg['Cc'] = ', '.join(cc_email)
  msg['Subject'] = subject

  msg.attach(MIMEText(body, 'plain'))

  if attachment_path and os.path.exists(attachment_path):
    with open(attachment_path, "rb") as attachment:
      part = MIMEBase('application', 'octet-stream')
      part.set_payload(attachment.read())
    encoders.encode_base64(part)
    part.add_header(
        'Content-Disposition',
        f"attachment; filename= {os.path.basename(attachment_path)}",
    )
    msg.attach(part)

  try:
    server = smtplib.SMTP_SSL('smtp.zoho.com', 465)
    server.login(sender_email, 'n6sZNfR')
    text = msg.as_string()
    server.sendmail(sender_email, receiver_email + [cc_email], text)
    server.quit()
    print("Email sent successfully!")
  except Exception as e:
    print(f"Error sending email: {e}")

In [ ]:
# @title applt the defined fuction to send mail
sender_email = 'asjidack@gmail.com'
receiver_email = ['aewgfrg@gmail.com','eefff@gmail.com','afeafdaef@gmail.com']
cc_email = ['fghfghc@gamil.com','siftx@gmail.com']
subject = 'Overall Product Analysis'
body = "Dear Team,\n\nI have attached Overall Product Analysis.\n\n Thanks & Regards. \nAsjid Ali\n Data Analyst , Data Team , Al Anees Electronics"
attachment_path = f'Product Analysis {fromd}-{tod}.xlsx'

send_email(sender_email, receiver_email, cc_email, subject, body, attachment_path)

Email sent successfully!


In [ ]:
# connecting to the google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# loading data to the google sheet without manualy importing
import gspread
from oauth2client.service_account import ServiceAccountCredentials

scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']


# Verify the path to your credentials file is correct and you have the correct file
SF=#json file from google cloud will be given here
credentials = ServiceAccountCredentials.from_json_keyfile_name(
    SF, scope)


gc = gspread.authorize(credentials)

sheet = gc.open_by_key('1z0VVX8bLIsGpZiYl')

worksheet = sheet.get_worksheet(0)

df10 = styled_df.data

# Convert timestamp values to strings because this data containing date
df10 = df10.applymap(lambda x: x.strftime('%Y-%m-%d %H:%M:%S') if pd.notna(x) and isinstance(x, pd.Timestamp) else (None if pd.isna(x) else x))

worksheet.clear()

worksheet.update([df10.columns.values.tolist()] + df10.values.tolist())

{'spreadsheetId': '1z0V4RpuQ3VX8bLIsGpZiYlzgrR_0IWyNNXkndpuqtUA',
 'updatedRange': 'Sheet1!A1:X10369',
 'updatedRows': 10369,
 'updatedColumns': 24,
 'updatedCells': 220824}